In [1]:
import json
import pandas as pd
import re
from fuzzywuzzy import process
import numpy as np

# Load Data

In [2]:
def expand_nested_column(df, nested_col, id_col):
    """
    Expands a specified nested column in a Pandas DataFrame while keeping other columns unchanged.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        nested_col (str): The name of the column containing nested dictionaries to expand.
        id_col (str): The unique identifier column to merge back after expansion.

    Returns:
        pd.DataFrame: A DataFrame with the nested column expanded.
    """
    # Ensure the nested column is a list of dictionaries
    df[nested_col] = df[nested_col].apply(lambda x: x if isinstance(x, dict) else {})
    
    # Convert nested dictionary column to a DataFrame
    df_expanded = pd.json_normalize(df[nested_col]).add_prefix(f"{nested_col}_")
    
    # Merge expanded columns back to the original DataFrame
    df_final = df.drop(columns=[nested_col]).merge(df_expanded, left_on=id_col, right_on=f"{nested_col}_id", how='left')
    
    # Remove duplicates based on the unique identifier column
    df_final = df_final.drop_duplicates(subset=[id_col])

    return df_final

In [3]:
# Load JSON data

with open("hist.json", "r") as f:
   history_df = json.load(f)
history_df = pd.DataFrame(history_df )

history_df = expand_nested_column(history_df, 'quiz', 'quiz_id')
history_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9 entries, 0 to 27
Data columns (total 57 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   id                         9 non-null      int64 
 1   quiz_id                    9 non-null      int64 
 2   user_id                    9 non-null      object
 3   submitted_at               9 non-null      object
 4   created_at                 9 non-null      object
 5   updated_at                 9 non-null      object
 6   score                      9 non-null      int64 
 7   trophy_level               9 non-null      int64 
 8   accuracy                   9 non-null      object
 9   speed                      9 non-null      object
 10  final_score                9 non-null      object
 11  negative_score             9 non-null      object
 12  correct_answers            9 non-null      int64 
 13  incorrect_answers          9 non-null      int64 
 14  source            

# Data Cleaning

In [4]:

def dropNull(df): 
    # Function to drop Null columns
    null_cols = df.columns[df.isnull().all()]
    print("No. of Null Columns : ", len(null_cols))
    if len(null_cols) > 0:
        print("Dropped Null Columns : ",list(null_cols))
        df.drop(columns=null_cols, inplace=True)
    else : print("No columns were dropped")
    return df
    
def clean_text(text):
    """
    Removes Lowercase, extra spaces, special characters, and numbers.
    """
    text = text.lower().strip()  # Convert to lowercase and strip spaces
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'[^a-z\s]', '', text)  # Remove special characters and numbers
    return text

def generate_standardized_name(df, column_name, threshold=90):
    """   
    Cleans text and applies fuzzy matching to standardize similar entries.
    """
    # Clean the column
    clean_col = f"clean_{column_name}"
    df[clean_col] = df[column_name].astype(str).apply(clean_text)

    # Get unique cleaned values
    unique_values = df[clean_col].unique()

    # Define mapping dictionary for standardization
    mapping = {}

    # Perform fuzzy matching
    for value in unique_values:
        if not mapping:  # First entry, add directly
            mapping[value] = value
            continue
        
        match_result = process.extractOne(value, mapping.keys())  # Find best match
        
        if match_result:  # Ensure we got a result
            match, score = match_result  
            mapping[value] = mapping[match] if score > threshold else value
        else:
            mapping[value] = value  # If no match, keep as is

    # Apply mapping to standardize column
    standardized_col = f"standardized_{column_name}"
    df[standardized_col] = df[clean_col].map(mapping)
    
    return df


def preprocess_data(df_original):
    """
    Function to clean the data 
    """
    df = df_original.copy()

    df = dropNull(df)
    
    # Convert date columns to datetime format
    date_cols = ['submitted_at', 'created_at', 'updated_at', 'started_at', 'ended_at']
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], utc=True) # COnverting everting to UTC without loss of generality
            df[col] = df[col].dt.tz_localize(None)  
    
    # Convert accuracy to numeric (0-1)
    df['accuracy'] = df['accuracy'].astype(str).str.rstrip('%').astype(float) / 100
    
    # Connvert rank_text to integer
    df['rank'] = df['rank_text'].str.extract(r'(\d+)$')
    df['rank'] = df['rank'].map(int)

    # Covert duration to numeric values (in mins)
    df[['minutes', 'seconds']] = df['duration'].str.split(':', expand=True).astype(int)
    df['duration'] = df['minutes'] + df['seconds']/60

    df = generate_standardized_name(df, "quiz_topic", threshold=90)
    df = generate_standardized_name(df, "quiz_title", threshold=90)

    numeric_cols = ['speed','final_score','negative_score','quiz_negative_marks','quiz_correct_answer_marks']
    df[numeric_cols] = df[numeric_cols].map(float)
    
    del_columns = ['rank_text','minutes', 'seconds','clean_quiz_topic','clean_quiz_title', 'quiz_title', 
                  'quiz_description', 'quiz_topic','quiz_time', 'quiz_is_published', 'quiz_created_at', 'quiz_updated_at',
                  'quiz_duration', 'quiz_end_time','quiz_shuffle', 'quiz_show_answers','quiz_lock_solutions', 'quiz_is_form', 
                  'quiz_show_mastery_option','quiz_is_custom', 'quiz_show_unanswered', 'quiz_ends_at','quiz_live_count', 
                  'quiz_coin_count','quiz_daily_date', 'quiz_max_mistake_count', 'quiz_reading_materials','source','type' ]
    print('Dropping unneccessary columns ...')
    print(del_columns)
    df.drop(columns=del_columns,inplace=True)
    
    return df

In [5]:
history_df = preprocess_data(history_df)

No. of Null Columns :  7
Dropped Null Columns :  ['quiz_name', 'quiz_difficulty_level', 'quiz_reading_material', 'quiz_quiz_type', 'quiz_banner_id', 'quiz_exam_id', 'quiz_lives']
Dropping unneccessary columns ...
['rank_text', 'minutes', 'seconds', 'clean_quiz_topic', 'clean_quiz_title', 'quiz_title', 'quiz_description', 'quiz_topic', 'quiz_time', 'quiz_is_published', 'quiz_created_at', 'quiz_updated_at', 'quiz_duration', 'quiz_end_time', 'quiz_shuffle', 'quiz_show_answers', 'quiz_lock_solutions', 'quiz_is_form', 'quiz_show_mastery_option', 'quiz_is_custom', 'quiz_show_unanswered', 'quiz_ends_at', 'quiz_live_count', 'quiz_coin_count', 'quiz_daily_date', 'quiz_max_mistake_count', 'quiz_reading_materials', 'source', 'type']


In [6]:
history_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9 entries, 0 to 27
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         9 non-null      int64         
 1   quiz_id                    9 non-null      int64         
 2   user_id                    9 non-null      object        
 3   submitted_at               9 non-null      datetime64[ns]
 4   created_at                 9 non-null      datetime64[ns]
 5   updated_at                 9 non-null      datetime64[ns]
 6   score                      9 non-null      int64         
 7   trophy_level               9 non-null      int64         
 8   accuracy                   9 non-null      float64       
 9   speed                      9 non-null      float64       
 10  final_score                9 non-null      float64       
 11  negative_score             9 non-null      float64       
 12  correct_answers 

In [7]:
history_df.iloc[0]

id                                                                      336497
quiz_id                                                                     51
user_id                                           YcDFSO4ZukTJnnFMgRNVwZTE4j42
submitted_at                                        2025-01-17 10:00:18.027000
created_at                                          2025-01-17 10:00:18.044000
updated_at                                          2025-01-17 10:00:18.044000
score                                                                      108
trophy_level                                                                 2
accuracy                                                                   0.9
speed                                                                    100.0
final_score                                                              105.0
negative_score                                                             3.0
correct_answers                                     

# Feature engineering

We can generate insightful features that will shed light on varius aspects of the analysis.

In [8]:
def compute_features(df):
    # 1. Total marks for quiz
    df['quiz_total'] = df['quiz_questions_count'] * df['quiz_correct_answer_marks']
    
    # 2. No. of questions attempted 
    df['questions_attempted'] = df['correct_answers'] + df['incorrect_answers']
    
    # 3. Total time taken for the quiz 
    df["total_time_taken"] = (df["ended_at"] - df["started_at"]).dt.total_seconds()
    
    # 4. Average response time for a question
    df['avg_response_time'] = df["total_time_taken"] / df['questions_attempted'].replace(0, np.nan)
    
    # 5. Normalized score (0-1)
    df["normalized_score"] = df["final_score"] / df["quiz_total"]
    
    # 6. Effective accuracy 
    df["effective_accuracy"] = (
        (df['correct_answers'] - (df['incorrect_answers'] * df['quiz_negative_marks'])
         / df['quiz_correct_answer_marks']) / df['questions_attempted']
    )
    
    # 7. Average Time Taken per Correct Answer
    df["time_per_correct"] = df["total_time_taken"] / df["correct_answers"].replace(0, np.nan)
    
    # 8. Average Time Taken per Incorrect Answer
    df["time_per_incorrect"] = df["total_time_taken"] / df["incorrect_answers"].replace(0, np.nan)
    
    # 9. Average Time Spent on Mistakes
    df["time_on_mistakes"] = (df["total_time_taken"] * df["incorrect_answers"]) / df["total_questions"]
    
    # 10. Mistake Correction Rate
    df["mistake_correction_rate"] = df["mistakes_corrected"] / df["initial_mistake_count"].replace(0, np.nan)
    
    # 11. Impact of negative marks
    df["penalty_impact"] = df["negative_score"] / (
        df["final_score"].abs() + df["negative_score"].abs()
    )
    
    # 12. Success Rate
    df["success_rate"] = df["correct_answers"] / df["questions_attempted"].replace(0, np.nan)
    
    # 13. Completion Rate
    df["completion_rate"] = df["questions_attempted"] / df["total_questions"]
    
    # 14. Quiz Difficulty Score
    def assign_difficulty(x):
        if x > 0.8:
            return 'Hard'
        elif 0.6 < x <= 0.8:
            return 'Medium'
        elif 0.3 < x <= 0.6:
            return 'Easy'
        else:
            return 'Very Easy'
    
    difficulty_mapping = {"Very Easy": 1, "Easy": 2, "Medium": 3, "Hard": 4}
    
    df["difficulty"] = df.groupby("quiz_id")["final_score"].transform(lambda x: x.mean()) / df["quiz_total"]
    df["difficulty"] = df["difficulty"].apply(assign_difficulty)
    df["difficulty"] = df["difficulty"].map(difficulty_mapping)
    
    # 15. Time Pressure Index
    df["quiz_time_pressure"] = (df["duration"] * 60) / df["quiz_questions_count"]
    
    return df


In [11]:
history_df = compute_features(history_df)
history_df.to_csv('clean_history_data.csv',index=False)
history_df.head(2)

id  quiz_id                       user_id            submitted_at  \
0  336497       51  YcDFSO4ZukTJnnFMgRNVwZTE4j42 2025-01-17 10:00:18.027   
3  336448        6  YcDFSO4ZukTJnnFMgRNVwZTE4j42 2025-01-17 09:47:44.042   

               created_at              updated_at  score  trophy_level  \
0 2025-01-17 10:00:18.044 2025-01-17 10:00:18.044    108             2   
3 2025-01-17 09:47:44.056 2025-01-17 09:47:44.056     92             1   

   accuracy  speed  ...  effective_accuracy  time_per_correct  \
0       0.9  100.0  ...               0.875         26.111111   
3       1.0  100.0  ...               1.000         25.913043   

   time_per_incorrect  time_on_mistakes mistake_correction_rate  \
0               235.0             21.15                    0.75   
3                 NaN              0.00                    1.00   

  penalty_impact  success_rate  completion_rate  difficulty  \
0       0.027778           0.9              0.3           1   
3       0.000000           1.0              1.0           4   

   quiz_time_pressure  
0            9.000000  
3           39.130435  

[2 rows x 43 columns]